In [1]:
import pandas as pd
dataset = pd.read_csv("CKD.csv")
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset.columns

#independent, dependent variable
independent = dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dependent = dataset[['classification_yes']]

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid = { 'criterion':['gini', 'entropy', 'log_loss'],
                'splitter':['best', 'random']
             }
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit=True, verbose=3, scoring='balanced_accuracy')
grid.fit(independent, dependent)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END .....criterion=gini, splitter=best;, score=0.610 total time=   0.0s
[CV 2/5] END .....criterion=gini, splitter=best;, score=0.943 total time=   0.0s
[CV 3/5] END .....criterion=gini, splitter=best;, score=0.967 total time=   0.0s
[CV 4/5] END .....criterion=gini, splitter=best;, score=0.990 total time=   0.0s
[CV 5/5] END .....criterion=gini, splitter=best;, score=0.963 total time=   0.0s
[CV 1/5] END ...criterion=gini, splitter=random;, score=0.890 total time=   0.0s
[CV 2/5] END ...criterion=gini, splitter=random;, score=0.990 total time=   0.0s
[CV 3/5] END ...criterion=gini, splitter=random;, score=0.967 total time=   0.0s
[CV 4/5] END ...criterion=gini, splitter=random;, score=0.980 total time=   0.0s
[CV 5/5] END ...criterion=gini, splitter=random;, score=0.983 total time=   0.0s
[CV 1/5] END ..criterion=entropy, splitter=best;, score=0.610 total time=   0.0s
[CV 2/5] END ..criterion=entropy, splitter=best;,

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'splitter': ['best', 'random']},
             scoring='balanced_accuracy', verbose=3)

In [7]:
y_pred = grid.predict(independent)

In [9]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(dependent, y_pred)

In [11]:
from sklearn.metrics import classification_report
cf_report = classification_report(dependent, y_pred)

In [13]:
print(cf_matrix)
print(cf_report)

[[150   0]
 [  0 249]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       150
           1       1.00      1.00      1.00       249

    accuracy                           1.00       399
   macro avg       1.00      1.00      1.00       399
weighted avg       1.00      1.00      1.00       399



In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_score(dependent, grid.predict_proba(independent)[:,1])

1.0

In [23]:
from sklearn.metrics import f1_score
f1_macro=f1_score(dependent,y_pred,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'splitter': 'random'}: 1.0


In [25]:
re = grid.cv_results_
table = pd.DataFrame.from_dict(re)

In [27]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013842,0.003171,0.011412,0.003278,gini,best,"{'criterion': 'gini', 'splitter': 'best'}",0.610000,0.943333,0.966667,0.99,0.962925,0.894585,0.143063,5
1,0.013923,0.005850,0.010258,0.003239,gini,random,"{'criterion': 'gini', 'splitter': 'random'}",0.890000,0.990000,0.966667,0.98,0.983333,0.962000,0.036794,3
2,0.007333,0.008060,0.011857,0.006969,entropy,best,"{'criterion': 'entropy', 'splitter': 'best'}",0.610000,0.970000,0.916667,0.99,1.000000,0.897333,0.146521,4
3,0.002003,0.004007,0.015888,0.007489,entropy,random,"{'criterion': 'entropy', 'splitter': 'random'}",0.990000,1.000000,0.953333,0.98,0.973129,0.979293,0.015848,1
4,0.011944,0.002602,0.006064,0.005496,log_loss,best,"{'criterion': 'log_loss', 'splitter': 'best'}",0.610000,0.953333,0.916667,0.99,1.000000,0.894000,0.145012,6
5,0.006937,0.006090,0.011982,0.004466,log_loss,random,"{'criterion': 'log_loss', 'splitter': 'random'}",0.933333,0.980000,0.983333,0.99,0.973129,0.971959,0.020064,2
